In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://repo.telematika.org/images/project/maartengr_bertopic_hua522d65d231be4e754d1e268b6f12fcb_31598_700x350_resize_q75_box.jpg)repo.telematika.org

In [ ]:
train = pd.read_csv('../input/nlp-with-disaster-tweets-cleaning-data/train_data_cleaning.csv')
train.head()

#Code by Darek Kleczec https://www.kaggle.com/thedrcat/commonlit-what-are-we-reading-about

In [ ]:
!pip install --upgrade pip
!pip install --upgrade numpy
!pip install --upgrade sentence_transformers
!conda install -c conda-forge hdbscan --y
!pip install bokeh
!pip install --upgrade bertopic[visualization]

In [ ]:
from bertopic import BERTopic

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import sklearn.manifold
import numpy as np
import pandas as pd
import random
random.seed(42)

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper
from bokeh.palettes import plasma, d3, Turbo256
from bokeh.plotting import figure
from bokeh.transform import transform
import bokeh.io
bokeh.io.output_notebook()

In [ ]:
import bokeh.plotting as bpl
import bokeh.models as bmo
bpl.output_notebook()

In [ ]:
#sub = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
test = pd.read_csv('../input/nlp-with-disaster-tweets-cleaning-data/test_data_cleaning.csv')
train = pd.read_csv('../input/nlp-with-disaster-tweets-cleaning-data/train_data_cleaning.csv')

train['set'] = 'train'
test['set'] = 'test'

combined = pd.concat([train, test], ignore_index=True)
combined.target.fillna(3, inplace=True)

texts = combined.text.values.tolist()
targets = combined.target.values.tolist()
sets = combined.set.values.tolist() #no submission file

model = SentenceTransformer('stsb-distilbert-base')
embeddings = model.encode(texts)
out = sklearn.manifold.TSNE(n_components=2).fit_transform(embeddings)

color_mapper = LinearColorMapper(palette='Plasma256', low=min(targets), high=max(targets))

#Semantic space with targets color map

In the first chart, I'd like to explore the semantic space of the texts and see if there is any correlation with the targets. The dots represent our texts, the color intensity corresponds to our target. I arbitrarily set the target value from texts coming from the test set to 3, so that I can distinguish them on the map.

In [ ]:
SETS = ['train', 'test']
MARKERS = ['circle', 'triangle']

list_x = out[:,0]
list_y = out[:,1]
desc = texts

source = ColumnDataSource(data=dict(x=list_x, y=list_y, desc=desc, targets=targets, dset=sets))
hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("(x,y)", "(@x, @y)"),
    ('desc', '@desc'),
    ('targets', '@targets'),
    ('dset', '@dset')
])

p = figure(plot_width=1200, plot_height=600, tools=[hover], title="First Look at the Data")
p.scatter('x', 'y', size=10, source=source, legend='dset', color={'field': 'targets', 'transform': color_mapper},
         marker=bokeh.transform.factor_mark('dset', MARKERS, SETS),)

bpl.show(p)

#Semantic space with topics

Now let's see if we can cluster together the texts with BERTopic topic model. It seems the model has a hard time discovering topics from the stories/prose, it's much easier for texts that appear to come from Wikipidia or similar sources. Also, there is a substantial percentage of outliers.

In [ ]:
model = BERTopic(language="english", min_topic_size=20)
topics, probs = model.fit_transform(texts)

topic_words = ['-1: outlier']
for i in range(len(set(topics))-1):
  tpc = model.get_topic(i)[:7]
  words = [x[0] for x in tpc]
  tw = ' '.join([str(i) + ':'] + words)
  topic_words.append(tw)

exp_topics = [topic_words[x+1] for x in topics]

clrs = random.sample(Turbo256, len(set(topics)))
color_map = bmo.CategoricalColorMapper(factors=topic_words, palette=clrs)

In [ ]:
list_x = out[:,0]
list_y = out[:,1]
desc = texts

source = ColumnDataSource(data=dict(x=list_x, y=list_y, desc=desc, topic=exp_topics, target=targets, dset=sets,))
hover = HoverTool(tooltips=[
    ("index", "$index"),
    ('desc', '@desc'),
    ('topic', '@topic'),
    ('target', '@target'),
    ('dset', '@dset'),
])

p = figure(plot_width=1200, plot_height=600, tools=[hover], title="Topics from BERTopic model")
p.scatter('x', 'y', size=10, source=source,
         fill_color=transform('topic', color_map),
         marker=bokeh.transform.factor_mark('dset', MARKERS, SETS),
         legend='dset'
)
# p.legend.location = "top_left"
# p.legend.click_policy="hide"

bokeh.plotting.show(p)

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thanks to Darek Kleczek @thedrcat for the script.' )